In [1]:
import Bio
import Bio.Cluster
from Bio import SeqIO
from Bio import AlignIO
from Bio import pairwise2
from scipy.spatial import distance
import os
import threading
from concurrent.futures import ThreadPoolExecutor
from concurrent.futures import ProcessPoolExecutor
from sklearn.cluster import SpectralClustering
from collections import Counter
from Bio.Cluster import distancematrix
from Bio.Phylo.TreeConstruction import DistanceCalculator
import numpy as np

In [2]:
DATA_PATH = "data/DQB_filtered_reads/"
workers=5


In [3]:
raw = []
allFiles = os.listdir(DATA_PATH)
def load(file):
    i = 0
    for record in SeqIO.parse((DATA_PATH + file), "fastq"):
        raw.append(record.seq)
        break


with ThreadPoolExecutor(max_workers=workers) as executor:
    executor.map(load, allFiles , chunksize=37)

sequences = Counter(raw)


In [12]:
x=sequences.keys()
c=sequences.values()
l= []
for i in  x:
    l.append(list(i))


In [5]:
c


dict_values([91, 18, 1, 1, 1, 11, 18, 1, 8, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [13]:
x

dict_keys([Seq('GGCCAGTGCTACTTCACCAACGGGACGGAGCGGGTGCGGGGTGTGGACAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGGGTGTGACCAGACAC...TGC'), Seq('TGCGAGGGCTACTTCACCAACGGGACGGAGCGGGTGCGGTACCTGAACAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGGGTGTGACCAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGGGTGTGGCCAGATAC...TGC'), Seq('TGCGAGTGCTACTTCACCAACGGGACGGAGCGGGTGCGGTACCTGAACAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGCTCCTGACCAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGGGTGTGACCAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGGGTGTGACCAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACGGAGCGGGTACGGGGTGTGGACAGATAC...TGC'), Seq('GGCCAGTGCTACTTCACCAACGGGACGGAGCGGGTGCGGGGTGTGGACAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGGTGCGGGGTGTGACCAGACGC...TGC'), Seq('GGCCAGTGCTACTTCACCAACGGGACGGAGCGGGTGCGGGGTGTGGACAGATAC...TGC'), Seq('GGCGAGTGCTACTTCACCAACGGGACAGAGCGGATGCGGGGTGTGACCAGATAC...TGC'), Seq('GGCCAGTGCTACTTCACC

In [10]:
n=len(c)
sim=np.zeros((n,n))

for i in range (0,len(l)):
    for j in range (0,i):
        sim[i][j]=distance.hamming(l[i],l[j])



In [11]:
for i in sim:
    print(i)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.12935323 0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
[0.10447761 0.11940299 0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.        ]
[0.10945274 0.039801   0.09950249 0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.

In [ ]:
alk= Bio.Cluster.kmedoids(sim, nclusters=10,npass=20)

In [ ]:
alk

In [22]:

sim=[[0 for i in range(len(l))] for j in range(len(l))]

for i in range (0,len(l)):
    for j in range (0,i):
        sim[i][j]=distance.hamming(l[i],l[j])



In [ ]:
for i in sim:
    print(i)

In [15]:
alk= Bio.Cluster.kmedoids(sim, nclusters=10,npass=20)

In [16]:
alk

(array([ 0,  1,  6,  8,  8,  5,  6,  8,  8,  6, 10,  0, 12,  8,  1, 15,  1,
         0,  0,  1,  1,  1,  1,  6, 24,  1,  1,  1,  1,  1,  1,  1, 32,  1],
       dtype=int32),
 0.1890547263681593,
 1)